In [1]:
from src.data._dataobject import DataObject
from src.data.types._datetime import DateTime
from src.data.types._text import String
from src.data.types._int import Int
from src.data.types._phone_number import PhoneNumber


In [2]:
class User(DataObject):
    first_name = String()
    last_name = String()
    mobile = PhoneNumber()
    id = Int()
    created = DateTime(default='now')

In [3]:
u = User(
    first_name='Frodo',
    last_name='Baggins',
    mobile='+39 375 57 44 512',
    id=1
    )